In [1]:
"""
To Make sure the Best Axon method works and also
to see if can decompose the axon any better 
(especially if can help with crosses and parts that aren't skeletonized             )

"""

'\nTo Make sure the Best Axon method works and also\nto see if can decompose the axon any better \n(especially if can help with crosses)\n\n'

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

In [6]:
import datajoint_utils as du
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-17 17:56:37,242 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-17 17:56:37,244 - settings - Setting database.user to celiib
INFO - 2021-02-17 17:56:37,244 - settings - Setting database.password to newceliipass
INFO - 2021-02-17 17:56:37,249 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-17 17:56:37,250 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-17 17:56:37,264 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-17 17:56:37,580 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-17 17:56:37,613 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-17 17:56:37,962 - settings - Setting enable_python_native_blobs to True


In [144]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import networkx_utils as xu
import datajoint as dj
import neuron_visualizations as nviz
import neuron
import neuron_utils as nru
import proofreading_utils as pru
import classification_utils as clu
import system_utils as su
from importlib import reload

# Checking the Double Axon Neurons

In [117]:
multi_axon_segs,multi_axon_splits = (minnie.AutoProofreadNeurons2() & "cell_type_predicted='excitatory'"
 & "n_axons>1").fetch("segment_id","split_index")

In [118]:
len(multi_axon_segs)

9346

In [ ]:
segment_id,split_index = (864691134947394812, 0) #was good for testing the high fidelity

In [719]:
curr_index = 
segment_id = multi_axon_segs[curr_index]
split_index = multi_axon_splits[curr_index]
segment_id,split_index

(864691135428517680, 0)

In [720]:
du.plot_proofread_neuron(segment_id,
                        split_index=split_index,
                        plot_errored_faces=True,
                        plot_synapses=True,
                        plot_nuclei=True,
                        plot_axon=True)

Synapsse Results:
Total Number of Synapses = 413
n_valid_syn_centers_presyn = 16
n_errored_syn_centers_presyn = 130
n_valid_syn_centers_postsyn = 267
n_errored_syn_centers_postsyn = 0
Including axon mesh


In [475]:
neuron_obj = du.decomposition_with_spine_recalculation(segment_id,
                                                      split_index=split_index)

Decompressing Neuron in minimal output mode...please wait
Pulled from Table Decomposition so setting split_index = 0
Number of Neurons found = 1


In [476]:
if type(neuron_obj) == tuple:
    test_neuron = neuron_obj[0][split_index]
else:
    test_neuron = neuron_obj

In [477]:
nviz.visualize_neuron(test_neuron,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [478]:
true_axons,true_axon_angles,axon_like_errors = clu.axon_classification(test_neuron,
                        return_axon_angles=True,
                       plot_candidates=False,
                       plot_axons=True,
                        best_axon=True,
                       verbose=True)


Part 1: Axon like branchese 
{'L0': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]), 'L1': array([ 1,  5, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]), 'L2': array([ 0,  3,  5,  9, 10, 11, 12, 13, 14, 18]), 'L3': array([ 5,  9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27,
       28, 29])}
Limb L0 soma angle: 151.55 
Limb L1 soma angle: 86.23 
Limb L2 soma angle: 115.25 
Limb L3 soma angle: 95.03 

Part 2: possible_axon_limbs = [0, 1, 2, 3]

Phase 2: Working on Limb L0
nodes_to_eliminate = []

Phase 2: Working on Limb L1
nodes_to_eliminate = [2 3 4]

Phase 2: Working on Limb L2
nodes_to_eliminate = [15]

Phase 2: Working on Limb L3
nodes_to_eliminate = [10]

----Working on Limb L0-----
No nodes were eliminated so don't need to add back any ca

In [67]:
# """
# Pseudocode: 
# if there even is an axon
# 1) Get the limb name of the best axon
# 2) Get the concept network of the limb and the starting node
# 3) Get the branches that would be axon
# 4) Delete the branches from the concept network
# 5) For each connected component in the leftover network
# - if not have starting node and all in axon-like: 
# add to list to add to the true axon

# 6) add the new nodes to the axon group
# """

# neuron_obj = test_neuron

# if len(best_limb_branch_dict) > 0:
    
#     #1) Get the limb name of the best axon
#     limb_name_of_axon = list(best_limb_branch_dict.keys())
#     if len(limb_name_of_axon) > 1:
#         raise Excpetion("More than 1 axon key")
    
#     limb_name_of_axon = limb_name_of_axon[0]
    
#     curr_limb = neuron_obj[limb_name_of_axon]
#     #2) Get the concept network of the limb and the starting node
#     conc_net = nx.Graph(curr_limb.concept_network)
#     start_node = curr_limb.current_starting_node
    
#     #3) Get the branches that would be axon
#     axon_branches = best_limb_branch_dict[limb_name_of_axon]
#     axon_like_branches = axon_like_limb_branch_dict[limb_name_of_axon]
    
#     #4) Delete the branches from the concept network
#     conc_net.remove_nodes_from(axon_branches)
    
    
#     #5) For each connected component in the leftover network
#     new_axon_branches = []
#     for conn_comp in nx.connected_components(conc_net):
#         conn_comp = list(conn_comp)
        
#         #- if not have starting node and all in axon-like: 
#         #add to list to add to the true axon
#         axon_like_in_conn_comp = np.intersect1d(axon_like_branches,conn_comp)
#         if start_node not in conn_comp and len(axon_like_in_conn_comp) == len(conn_comp):
#             new_axon_branches += conn_comp
    
    
        
#     if len(new_axon_branches) > 0:
#         best_limb_branch_dict[limb_name_of_axon] = np.array(list(axon_branches) + new_axon_branches)
        
# best_limb_branch_dict

{'L6': array([ 8,  9, 11, 13, 18, 20, 21, 22, 23, 24, 25, 26, 27, 15, 17, 12,  0,
         1,  2,  3,  4,  5,  6,  7])}

# Getting a more high fidelity axon skeleton

In [804]:
curr_index = 3010
segment_id = multi_axon_segs[curr_index]
split_index = multi_axon_splits[curr_index]
segment_id,split_index

(864691135539149252, 1)

In [821]:
segment_id=864691135539149252
split_index = 0

In [822]:
minnie.AutoProofreadNeurons2() & dict(segment_id = segment_id,
                                      split_index=split_index)

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,proof_version the version of code used for this cell typing classification,multiplicity the total number of neurons that came from the parent segment id,cell_type_predicted morphology predicted by classifier,spine_category,n_axons Number of axon candidates identified,n_apicals Number of apicals identified,axon_length length (in um) of the classified axon skeleton,axon_area # area (in um^2) of the classified axon,nucleus_id id of nucleus from the flat segmentation Equivalent to Allen: 'id'.,nuclei_distance the distance to the closest nuclei (even if no matching nuclei found),n_nuclei_in_radius the number of nuclei within the search radius of 15000 belonging to that segment,n_nuclei_in_bbox the number of nuclei within the bounding box of that soma,"soma_x x coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_y y coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_z z coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)",max_soma_n_faces The largest number of faces of the somas,max_soma_volume The largest volume of the somas the (volume in billions (10*9 nm^3)),n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,axon_angle_maximum the anlge of an identified axon,spine_density_classifier the number of spines divided by skeletal length for branches analyzed in classification,n_branches_processed the number branches used for the spine density analysis,skeletal_length_processed The total skeletal length of the viable branches used for the spine density analysis,n_branches_in_search_radius the number branches existing in the search radius used for spine density,skeletal_length_in_search_radius The total skeletal length of the branches existing in the search radius used for spine density,run_time the amount of time to run (seconds)
864691135539149252,0,0.25,30.00,0,0,0,1,2,excitatory,densely_spined,1,1,60.279630386873905,110.77288794537196,208310,941.61,1,1,145152,264171,2528

In [823]:
du.plot_proofread_neuron(segment_id,
                        split_index=split_index,
                        plot_errored_faces=True,
                        plot_synapses=True,
                        plot_nuclei=True,
                        plot_axon=True)

Synapsse Results:
Total Number of Synapses = 378
n_valid_syn_centers_presyn = 0
n_errored_syn_centers_presyn = 230
n_valid_syn_centers_postsyn = 133
n_errored_syn_centers_postsyn = 15
Including axon mesh


In [824]:
neuron_obj = du.decomposition_with_spine_recalculation(segment_id,
                                                      split_index=split_index)

Decompressing Neuron in minimal output mode...please wait
Number of Neurons found = 1


In [825]:
if type(neuron_obj) == tuple:
    neuron_obj = neuron_obj[0][split_index]

In [829]:
neuron_obj.segment_id

864691135539149252

In [826]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [827]:
print("hi")

hi


In [828]:
clu.axon_classification(neuron_obj,
                        plot_candidates=True,
                       best_axon=True,
                       plot_axons=True)


Part 1: Axon like branchese 
{'L0': array([2, 3, 5, 6, 7]), 'L3': array([0, 1, 2]), 'L4': array([ 0,  2,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17, 18])}
Limb L0 soma angle: 7.19 
Limb L1 soma angle: 74.47 
Limb L2 soma angle: 134.1 
Limb L3 soma angle: 101.19 
Limb L4 soma angle: 94.79 

Part 2: possible_axon_limbs = [1, 2, 3, 4]

Phase 2: Working on Limb L1
nodes_to_eliminate = [3]

Phase 2: Working on Limb L2
nodes_to_eliminate = []

Phase 2: Working on Limb L3
nodes_to_eliminate = [0]

Phase 2: Working on Limb L4
nodes_to_eliminate = []

----Working on Limb L1-----
No endpoints were eliminated so don't need to add back any candidates

----Working on Limb L2-----
No nodes were eliminated so don't need to add back any candidates

----Working on Limb L3-----
No endpoints were eliminated so don't need to add back any candidates

----Working on Limb L4-----
No nodes were eliminated so don't need to add back any candidates

 Working on visualization type: mesh


Final Candidates

Limb 1
Candidate 0 (crimson): array([0, 1, 2])
Candidate 1 (gold): array([4])

Limb 2
Candidate 0 (blue): array([0, 1, 2])

Limb 3
Candidate 0 (gold): array([1])
Candidate 1 (darkslategray): array([2])

Limb 4
Candidate 0 (sandybrown): array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])
Part 3: axon_subgraph_candidates = {1: [[0, 1, 2], [4]], 2: [[0, 1, 2]], 3: [[1], [2]], 4: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]]}


 --- Working on limb 1, candidate # 0
0 out of 3 branches are axons
Axon percentage = 0.0
Not adding candidate no axon branches detected 


 --- Working on limb 1, candidate # 1
0 out of 1 branches are axons
Axon percentage = 0.0
Not adding candidate no axon branches detected 


 --- Working on limb 2, candidate # 0
0 out of 3 branches are axons
Axon percentage = 0.0
Not adding candidate no axon branches detected 


 --- Working on limb 3, candidate # 0
1 out of 1 branches are axons
Axon

({},
 {'L0': array([2, 3, 5, 6, 7]),
  'L3': array([0, 1, 2]),
  'L4': array([ 0,  2,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17, 18])})

# Should not be absolutely tiny at the start

In [ ]:
"""
may have to worry about the width array

"""

In [665]:
neuron_obj.axon_mesh, neuron_obj.axon_starting_coordinate

(<trimesh.Trimesh(vertices.shape=(4778, 3), faces.shape=(9544, 3))>,
 array([1063610.48147194, 1014205.10564622,  652002.57828337]))

In [666]:
verbose = True

axon_limb_name = neuron_obj.axon_limb_name
curr_limb = neuron_obj[axon_limb_name]
axon_starting_branch = neuron_obj.axon_starting_branch
axon_starting_branch_mesh = curr_limb[axon_starting_branch].mesh
axon_starting_coordinate = neuron_obj.axon_starting_coordinate

upstream_node_to_axon_starting_branch = None

if axon_starting_branch != curr_limb.current_starting_node:
    if verbose:
        print(f"Starting axon branch was not the limb starting node so generating border vertices and computing upstream branch")
    border_vertices_for_axon = tu.largest_border_to_coordinate(
        mesh = axon_starting_branch_mesh,
        coordinate =axon_starting_coordinate,
        distance_threshold = 1000,
        plot_border_vertices = False,
        error_on_no_border = True,
        plot_winning_border = False,
        verbose = False)
    
    upstream_node_to_axon_starting_branch = xu.upstream_node(curr_limb.concept_network_directional,
                                                             axon_starting_branch
                                                               )
    
else:
    if verbose:
        print(f"Starting axon branch was so using border vertices and NOT computing upstream branch")
    border_vertices_for_axon = curr_limb.current_touching_soma_vertices
    upstream_node_to_axon_starting_branch = None

soma_touching_vertices_dict = {0:[border_vertices_for_axon]}

if verbose:
    print(f"upstream_node_to_axon_starting_branch = {upstream_node_to_axon_starting_branch}")

Starting axon branch was so using border vertices and NOT computing upstream branch
upstream_node_to_axon_starting_branch = None


In [667]:
# divide the meshes into the only big continuous one (because if disconnected probably)
diff_meshes = tu.split_by_vertices(neuron_obj.axon_mesh)
axon_mesh_filtered = tu.filter_meshes_by_containing_coordinates(diff_meshes,
                                          nullifying_points=border_vertices_for_axon[0],
                                           method="distance",
                                           distance_threshold = 0,
                                          filter_away=False)[0]
if verbose:
    if len(diff_meshes) > 1:
        print(f"More than 1 seperated mesh (taking the top one): {diff_meshes}")

In [668]:
limb_correspondence_individual,concept_network = pre.preprocess_limb(axon_mesh_filtered,
                   soma_touching_vertices_dict = soma_touching_vertices_dict,
                    meshparty_segment_size = 100,
                    combine_close_skeleton_nodes=True,
                    #combine_close_skeleton_nodes_threshold=1200,
                    combine_close_skeleton_nodes_threshold_meshparty = 1300,
                    filter_end_node_length_meshparty = 1500,
                    filter_end_node_length = 1500,
                                                                     
                    use_meshafterparty=False,
                    perform_cleaning_checks = True,
                    
                    
                    #concept_network parameters
                    run_concept_network_checks = True,
                    return_concept_network = True,
                    return_concept_network_starting_info=False,
                    
                    #printing controls
                    verbose = True,
                    print_fusion_steps=True,
                    
                    check_correspondence_branches = True,
                    filter_end_nodes_from_correspondence=True,
                    error_on_no_starting_coordinates=True,
                    invalidation_d=1200
                                                                     
                    
                   )

  0%|          | 0/1 [00:00<?, ?it/s]

soma_touching_vertices_dict = {0: [TrackedArray([[1063992. , 1014538. ,  652499.2],
              [1063883. , 1014418. ,  652506.8],
              [1063847. , 1014525. ,  652427.1],
              [1063912. , 1014539. ,  652256.2],
              [1063917. , 1014304. ,  652426.2],
              [1064028. , 1014342. ,  652029. ],
              [1063830. , 1013944. ,  652112.4],
              [1063991. , 1014069. ,  651947.2],
              [1063932. , 1014190. ,  652328.9],
              [1063906. , 1014197. ,  651976.1],
              [1064022. , 1013920. ,  651980.4],
              [1063818. , 1014339. ,  652030.6],
              [1063881. , 1014006. ,  651987. ],
              [1064036. , 1014629. ,  652369.3],
              [1063983. , 1014025. ,  652180.2],
              [1063921. , 1014431. ,  652134.6],
              [1064070. , 1014483. ,  652165.3],
              [1064062. , 1014280. ,  651954.9]])]}
root_curr = [1063992.  1014538.   652499.2]
Time for preparing soma vertices and

100%|██████████| 1/1 [00:00<00:00,  8.31it/s]

meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 0.2685394287109375
branches_touching_root = [12]
combining close nodes
combine_threshold = 1300
length of Graph = 1315
Working on path [335. 339. 342. 345. 349. 353. 359. 361.]
path_degrees = [3, 2, 2, 2, 2, 2, 2, 3]
Working on path [ 355.  358.  360. 1315.]
path_degrees = [3, 2, 2, 4]
Working on path [553. 554. 557.]
path_degrees = [3, 2, 3]
Working on path [888. 890. 892. 893.]
path_degrees = [3, 2, 2, 3]
Working on path [939. 941. 943. 945. 946.]
path_degrees = [3, 2, 2, 2, 3]
After combining close endpoints max(kept_branches_idx) = 38, len(kept_branches_idx) = 34
empty_indices % = 0.0
 conflict_indices % = 0.0713308784850079



AFTER face_lookup_resolved_test



Decomposing first pass: 0.8263804912567139
Only Using MeshParty Skeletonization and Mesh Correspondence
Another print
Divinding into MP and MAP pieces: 2.7894973754882812e-05
Total time for MAP sublimb processing 3.0994415283203125e-06
---- Working on MP Decomposition #0 ----
Using Quicker soma_to_piece_touching_vertices because no MAP and only one sublimb_mesh piece 
MP filtering soma verts: 4.363059997558594e-05
Fixing Possible Soma Extension Branch for Sublimb 0
Total time for mesh KDTree = 0.01119232177734375
sbv[0].reshape(-1,3) = [[1063992.  1014538.   652499.2]]
closest_sk_pt_coord BEFORE = [1063610.4813302  1014205.10531349  652002.5778974 ]
current_skeleton.shape = (23, 2, 3)
node_for_stitch = 23: [1063610.4813302  1014205.10531349  652002.5778974 ]
node_for_stitch AFTER = 23: [1063610.4813302  1014205.10531349  652002.5778974 ]
possible_node_loc = [ 0 23]
possible_node_loc AFTER = [ 0 23]
curr_shortest_path = [23]
Current stitch point was a branch or endpoint
change_status f


Removed 4 skeletal branches



9543 9544
For marked faces: None
empty_indices % = 0.0
 conflict_indices % = 0.0



AFTER face_lookup_resolved_test


In [669]:
concept_network
curr_concept_network = concept_network[0][0]
curr_concept_network

In [670]:
starting_node = xu.get_starting_node(curr_concept_network,only_one=False)[0]

starting_coordinate = curr_concept_network.nodes[starting_node]["starting_coordinate"]
touching_soma_vertices = curr_concept_network.nodes[starting_node]["touching_soma_vertices"]

nviz.plot_limb_correspondence(limb_correspondence_individual,
                             scatters = [starting_coordinate],
                             scatter_size=1)

# nviz.plot_objects(skeletons=[k["branch_skeleton"] for k in limb_correspondence_individual.values()],
#                  scatters=[starting_coordinate],
#                  scatters_colors="red")

# Adding the part that connects back to the original mesh connection point

In [671]:
"""
Psuedocode: 
1) Get the starting axon branch and the upstream node
2) Find the common endpoint
3) Add a skeletal branch from starting_coordinate (new)
and the common endpoint to the starting node of new
axon decomposition

"""
verbose = True
import numpy_utils as nu

#1) Get the starting axon branch and the upstream node
if upstream_node_to_axon_starting_branch is not None:
    print("Readjusting starting axon branch skeleton")
    
    #1) Get the starting axon branch and the upstream node
    axon_starting_branch,upstream_node_to_axon_starting_branch
    
    #2) Find the common endpoint
#     shared_endpoint = shared_skeleton_endpoints_for_connected_branches(neuron_obj[axon_limb_name],
#                                                 axon_starting_branch,
#                                                      upstream_node_to_axon_starting_branch,
#                                                 verbose=False)
    shared_endpoint = neuron_obj.axon_starting_coordinate
    
    if shared_endpoint.ndim > 1:
        shared_endpoint = shared_endpoint[0]
    
    #3) Add a skeletal branch from starting_coordinate (new)
    #and the common endpoint to the starting node of new
    #axon decomposition
    
    curr_endpoints = sk.find_skeleton_endpoint_coordinates(limb_correspondence_individual[starting_node]["branch_skeleton"])
    if len(nu.matching_rows(curr_endpoints,axon_starting_coordinate)) == 0:
        print("Fixing the axon starting branch endpoint to align with upstream branch")
        
        skeleton_pre_fix = limb_correspondence_individual[starting_node]["branch_skeleton"]
        new_skeleton_segment = np.array([starting_coordinate,shared_endpoint]).reshape(-1,2,3)

        limb_correspondence_individual[starting_node]["branch_skeleton"] = sk.stack_skeletons([skeleton_pre_fix,
                                                                                               new_skeleton_segment
                                                                                             ])
        if verbose:
            print(f"Starting Branch {starting_node} skeleton before fix: {skeleton_pre_fix.shape}")
            print(f"Starting Branch {starting_node} skeleton AFTER fix: {limb_correspondence_individual[starting_node]['branch_skeleton'].shape}")
    else:
        if verbose:
            print("Not attempting to fix the limb correspondence because the axon_starting_coordainte was already an endpoint")
    
    plot_connecting_skeleton_fix = False
    if plot_connecting_skeleton_fix:
        upstream_branch = neuron_obj[axon_limb_name][upstream_node_to_axon_starting_branch]
        nviz.plot_objects(meshes=meshes + [upstream_branch.mesh],
                          meshes_colors="random",
                          skeletons=skeletons +  [upstream_branch.skeleton],
                          skeletons_colors="random",
                                      scatters=[axon_starting_coordinate],
                                     scatter_size=0.3)
else:
    if verbose:
        print(f"Upstream node was None so don't have to adjust")

Upstream node was None so don't have to adjust


# MAY HAVE TO WORRY ABOUT COMPUTING THE WIDTH ARRAY

In [695]:
nviz.plot_objects(meshes=[base_branch.mesh,neuron_obj["S0"].mesh],
                 skeletons=[base_branch.skeleton],
                 scatters=[neuron_obj.axon_starting_coordinate],
                 scatter_size=0.5)

In [688]:
(base_final_skeleton,
    base_final_widths,
    base_final_seg_lengths)

(array([[[ 812911.15291954, 1023678.44823879,  683685.57828298],
         [ 812580.08744052, 1024041.84340991,  684231.26977554]],
 
        [[ 812580.08744052, 1024041.84340991,  684231.26977554],
         [ 812096.77348479, 1024586.88844956,  684916.15709356]],
 
        [[ 812096.77348479, 1024586.88844956,  684916.15709356],
         [ 811647.95423419, 1025112.89966179,  685638.16443364]]]),
 array([ 91.54393304,  95.27527144, 103.01633192]),
 array([734.46552461, 999.86854808, 999.71051522]))

In [698]:
common_endpoint

array([ 812911.15291954, 1023678.44823879,  683685.57828298])

In [705]:
base_branch = neuron_obj[3][0]
common_endpoint = neuron_obj.axon_starting_coordinate

(base_final_skeleton,
    base_final_widths,
    base_final_seg_lengths) = nru.align_and_restrict_branch(base_branch,
                              common_endpoint=common_endpoint,
                             offset=0,
                             comparison_distance=2000,
                             skeleton_segment_size=1000,
                              verbose=True,
                             )
base_final_widths

Base needs flipping
Skeleton after flip = [[[ 812911.15291954 1023678.44823879  683685.57828298]
  [ 812589.90816659 1024030.87496901  684216.29490847]]

 [[ 812589.90816659 1024030.87496901  684216.29490847]
  [ 812106.14499027 1024575.83680895  684902.37173448]]

 [[ 812106.14499027 1024575.83680895  684902.37173448]
  [ 811656.57782285 1025103.47194025  685624.10200906]]

 [[ 811656.57782285 1025103.47194025  685624.10200906]
  [ 811195.38272902 1025634.94365143  686333.93652663]]

 [[ 811195.38272902 1025634.94365143  686333.93652663]
  [ 810666.86509697 1026335.02397587  686803.68181199]]

 [[ 810666.86509697 1026335.02397587  686803.68181199]
  [ 809968.70773916 1027014.83541978  687025.31665159]]

 [[ 809968.70773916 1027014.83541978  687025.31665159]
  [ 809248.92550413 1027661.98930973  687273.90622514]]

 [[ 809248.92550413 1027661.98930973  687273.90622514]
  [ 808588.1503902  1028324.74290517  687628.74399434]]

 [[ 808588.1503902  1028324.74290517  687628.74399434]
  [ 807

array([ 91.54393304,  95.27527144, 103.01633192])

In [704]:
np.mean(base_final_widths)

96.6118454688276

In [708]:
restr_skeleton = sk.restrict_skeleton_to_distance_from_coordinate(neuron_obj[0].skeleton,
                                                 neuron_obj[0].current_starting_coordinate,
                                                 10000,)
nviz.plot_objects(main_skeleton=neuron_obj[0].skeleton,
                 skeletons=[restr_skeleton],
                 skeletons_colors="red")          

In [715]:
[k for k in dir(nru) if "endpoint" in k]

['classify_endpoint_error_branches_from_limb_concept_network',
 'ordered_endpoints_on_branch_path',
 'shared_skeleton_endpoints_for_connected_branches']

In [718]:
branches

array([0])

In [735]:


axon_limb_dict = neuron_obj.axon_limb_branch_dict
axon_limb_name = list(axon_limb_dict.keys())[0]

limb_obj = neuron_obj[axon_limb_name]
branches = axon_limb_dict[axon_limb_name]

nru.closest_branch_endpoint_to_limb_starting_coordinate(limb_obj,
                                                    branches,
                                                       )


array([1435895.68871823,  637786.35929307, 1107653.21407107])

In [731]:
neuron_obj.axon_starting_coordinate

array([1435895.68871823,  637786.35929307, 1107653.21407107])

In [712]:
final_sk = sk.restrict_skeleton_from_start(neuron_obj[0].skeleton,
                                 cutoff_distance=10000)

In [714]:
nviz.plot_objects(main_skeleton=neuron_obj[0].skeleton,
                 skeletons=[final_sk[0]],
                 skeletons_colors="red")          